#📌 lIMPIEZA DE DATOS DE BOOKING ATRACCIONES
Este notebook carga datos de un archivo CSV en este casollamado `atracciones_barcelona2.csv`. Luego, extrae el precio promedio de una columna llamada "price" y lo guarda en una nueva columna llamada "price_avg".

Además, el código itera sobre las filas del DataFrame y, si las columnas "review_score" o "review_count" están vacías, intenta obtener estos datos de la URL proporcionada en la columna "url" utilizando web scraping con `requests` y `BeautifulSoup`. Finalmente, guarda el DataFrame actualizado en un nuevo archivo CSV llamado `booking_barcelona_actualizado.csv`.

##📌 LIBRERIAS

In [ ]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time

##📌 DESARROLLO

In [ ]:
df = pd.read_csv("atracciones_barcelona2.csv")

In [ ]:
def extraer_promedio(precio_str):
    if pd.isna(precio_str):
        return None
    numeros = [float(x.replace(',', '.')) for x in re.findall(r"\d+(?:[.,]\d+)?", precio_str)]
    if len(numeros) == 0:
        return None
    elif len(numeros) == 1:
        return numeros[0]
    else:
        return sum(numeros) / len(numeros)

In [ ]:
df["price_avg"] = df["price"].apply(extraer_promedio)

In [ ]:
def obtener_datos_desde_url(url):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")

        # 🔧 Debes ajustar las clases según el HTML real de Booking
        score_element = soup.find("span", class_="da8a6fe12c css-35ezg3")  # ejemplo
        count_element = soup.find("span", class_="da8a6fe12c")  # ejemplo

        score = float(score_element.text.strip().replace(",", ".")) if score_element else 0
        count_match = re.search(r'\d+', count_element.text.strip()) if count_element else None
        count = int(count_match.group()) if count_match else 0

        return score, count

    except Exception as e:
        print(f"Error accediendo a {url}: {e}")
        return 0, 0

## 🔄 Actualización y Guardado del DataFrame

Este código itera sobre el DataFrame, utiliza la función de web scraping para obtener datos faltantes de revisión, actualiza el DataFrame con la información obtenida y finalmente guarda el DataFrame actualizado en un nuevo archivo CSV.

In [ ]:
for idx, row in df.iterrows():
    if pd.isna(row["review_score"]) or pd.isna(row["review_count"]):
        print(f"Revisando URL: {row['url']}")
        score, count = obtener_datos_desde_url(row["url"])

        if pd.isna(row["review_score"]):
            df.at[idx, "review_score"] = score
        if pd.isna(row["review_count"]):
            df.at[idx, "review_count"] = count

        time.sleep(1)

df.to_csv("booking_barcelona_actualizado.csv", index=False)
print("Base actualizada guardada como 'booking_main_actualizado.csv'")

Revisando URL: https://www.booking.com/attractions/es/prbspnfdkbkw-admission-to-sagrada-familia.es.html?adplat=www-index-web_shell_header-attraction-missing_creative-1zwEmyAV9Sm0av9pzjIUKp&label=gen173nr-1BCAEoggI46AdIM1gEaEaIAQGYAQq4ARfIAQzYAQHoAQGIAgGoAgO4AuSv-sMGwAIB0gIkMzQzOWUzMjItNGVmMC00NmVjLTg0NTAtMWVlMGI1OWMzMzU02AIF4AIB&distribution_id=1zwEmyAV9Sm0av9pzjIUKp&aid=304142&client_name=b-web-shell-bff&source=searchresults-product-card&start_date=2025-07-21&end_date=2026-01-16&date=2025-07-21&ufi=-372490
Revisando URL: https://www.booking.com/attractions/es/prp9at1egmyc-park-guell-admission-ticket.es.html?adplat=www-index-web_shell_header-attraction-missing_creative-1zwEmyAV9Sm0av9pzjIUKp&label=gen173nr-1BCAEoggI46AdIM1gEaEaIAQGYAQq4ARfIAQzYAQHoAQGIAgGoAgO4AuSv-sMGwAIB0gIkMzQzOWUzMjItNGVmMC00NmVjLTg0NTAtMWVlMGI1OWMzMzU02AIF4AIB&distribution_id=1zwEmyAV9Sm0av9pzjIUKp&aid=304142&client_name=b-web-shell-bff&source=searchresults-product-card&start_date=2025-07-21&end_date=2026-01-16&dat